# HMDB database importer

In [19]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import ast
import re

import bisect
from rapidfuzz import process, fuzz

import sys
sys.path.append("src")

from hmdb_local_tools import  multiple_query, approximate_lookup

## Read compiled DB

In [20]:
dbname = "serum"
nmrdb = pd.read_csv("inst/spectral1hnmr.csv")
df = pd.read_csv(f"inst/{dbname}_metabolites_with_spectra.csv")

# Convert ppm from string to np.array
nmrdb['ppm'] = nmrdb['ppm'].apply(lambda x: np.array(ast.literal_eval(x.replace("'",''))))

def h_converter(x):
    x = np.array(ast.literal_eval(x.replace("'",'')))
    return x / np.sum(x) 
    
nmrdb['heights'] = nmrdb['heights'].apply(lambda x: h_converter(x))

## Prepare the dataset for approximate lookup using the metabolyte names and synomyms
df['synonyms'] = df['synonyms'].apply(lambda x: ast.literal_eval(x))
df['synonyms_cat'] = df['name'] + " " + df['synonyms'].str.join('')
df['synonyms_cat'] = df['synonyms_cat'].str.lower()

## Test approximage lookup

In [35]:
matches = approximate_lookup(df, 'synonyms_cat', 'Acetone', fuzz.partial_token_ratio, fuzz.ratio, limit=3)
print(matches)

[['Acetone' 'Acetone' 443 100.0]
 ['Butanone' 'Methylacetone' 204 70.0]
 ['Succinylacetone' 'Succinylacetone' 246 63.63636363636363]]


In [37]:
df.loc[443, 'description']

'Acetone, or propanone, is an organic compound with the formula (CH3)2CO. It is the simplest and smallest ketone. It is a colourless, highly volatile and flammable liquid with a characteristic pungent odour. Acetone is miscible with water and serves as an important organic solvent in its own right, in industry, home, and laboratory. Acetone is produced and disposed of in the human body through normal metabolic processes. It is normally present in blood and urine. People with diabetic ketoacidosis produce it in larger amounts. Acetone is not regarded as a waste product of metabolism. However, its physiological role in biochemical machinery is not clear. A model for the role of acetone metabolism is presented that orders the events occurring in acetonemia in sequence: in diabetic ketosis or starvation, ketone body production (b-hydroxy-butyrate, acetoacetate) provides fuel for vital organs (heart, brain, among others) raising the chance of survival of the metabolic catastrophe. However, 

## Rank matches according to multiplet similarity

In [38]:
query = [ {'range': (1.8, 1.86), 'mult': '*',},]

result = multiple_query(query, nmrdb, df)
result.head(10) 

,accession,name,similarity
17,HMDB0000661,Glutaric acid,1.500000
5,HMDB0000262,Thymine,1.500000
32,HMDB0001645,L-Norleucine,1.250000
18,HMDB0000694,L-2-Hydroxyglutaric acid,1.250000
29,HMDB0000991,DL-2-Aminooctanoic acid,1.250000
15,HMDB0000590,Glutarylglycine,1.250000
2,HMDB0000214,Ornithine,1.200000
0,HMDB0000070,Pipecolic acid,1.166667
35,HMDB0003072,Quinic acid,1.166667
19,HMDB0000716,L-Pipecolic acid,1.166667


In [33]:
query = [ {'range': (2.22, 2.29), 'mult': 's', 'ppm': np.array([2.26]), 'heights': np.array([1.0])},]

result = multiple_query(query, nmrdb, df)
result.head(10) 

,accession,name,similarity
37,HMDB0001659,Acetone,3.000000
1,HMDB0000060,Acetoacetic acid,2.500000
39,HMDB0001858,p-Cresol,2.333333
53,HMDB0011635,p-Cresol sulfate,2.250000
43,HMDB0002048,m-Cresol,2.200000
23,HMDB0000706,Aspartylphenylalanine,2.166667
50,HMDB0004999,Loperamide,2.111111
41,HMDB0001913,Omeprazole,2.111111
4,HMDB0000244,Riboflavin,2.100000
51,HMDB0005018,Amlodipine,2.083333


In [41]:
nmrdb[nmrdb['accession'] == 'HMDB0001645']

,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
148,HMDB0001645,m01,0.905,28,['7.14'],['9'],t,0.891,0.919,"[0.893, 0.905, 0.917]","[535.5, 542.7, 549.8]","[0.23135299219427577, 0.523417172593235, 0.245...",[9]
466,HMDB0001645,m02,1.353,39,[''],"['8', '7']",m,1.317,1.394,"[1.317, 1.323, 1.331, 1.335, 1.344, 1.356, 1.3...","[789.5, 793.0, 797.9, 800.5, 805.3, 812.6, 819...","[0.02782482458262763, 0.04814904427776434, 0.0...","[8, 7]"
814,HMDB0001645,m03,1.854,21,[''],['6'],m,1.811,1.907,"[1.811, 1.814, 1.819, 1.827, 1.831, 1.835, 1.8...","[1085.5, 1087.5, 1090.5, 1094.9, 1097.4, 1100....","[0.016255757247358442, 0.022758060146301815, 0...",[6]
2055,HMDB0001645,m04,3.739,12,"['6.92', '5.38']",['4'],dd,3.727,3.750,"[3.729, 3.737, 3.74, 3.749]","[2234.9, 2240.2, 2241.7, 2247.2]","[0.23729314420803788, 0.27718676122931446, 0.2...",[4]


In [4]:
query = [ {'range': (3.87, 3.93), 'mult': 'dd', 'ppm': np.array([3.8892, 3.8930, 3.9097, 3.9134]), 'heights': np.array([0.25, 0.25, 0.25, 0.25])},]

result = multiple_query(query, nmrdb, df)
result.head(10) 

,accession,name,similarity
7,HMDB0000122,D-Glucose,2.126998
15,HMDB0000191,L-Aspartic acid,1.965755
42,HMDB0000884,Ribothymidine,1.952838
35,HMDB0000660,D-Fructose,1.942185
33,HMDB0000609,DL-Dopa,1.941478
20,HMDB0000258,Sucrose,1.905719
38,HMDB0000742,Homocysteine,1.898036
66,HMDB0002006,"2,3-Diaminopropionic acid",1.894190
67,HMDB0002545,Galacturonic acid,1.865068
22,HMDB0000296,Uridine,1.828641


In [5]:
query = [{'range': (2.5, 2.6), 'mult': 'd'}, {'range': (2.6, 2.7), 'mult': 'd'}]

result = multiple_query(query, nmrdb, df)
result.head()

,accession,name,similarity
1,HMDB0000094,Citric acid,2.000000
3,HMDB0000402,2-Isopropylmalic acid,1.400000
4,HMDB0000736,Isobutyryl-L-carnitine,0.722222
7,HMDB0001257,Spermidine,0.666667
8,HMDB0001844,Methylsuccinic acid,0.500000


In [6]:
query = [{'range':(1.25, 1.35), 'mult': 'd'}, {'range': (4.05, 4.15), 'mult': 'q'}]

result = multiple_query(query, nmrdb, df)
result.head(6)    

,accession,name,similarity
1,HMDB0000190,L-Lactic acid,2.000000
0,HMDB0000030,Biotin,0.700000
7,HMDB0005000,Loratadine,0.642857
4,HMDB0000701,Hexanoylglycine,0.400000
3,HMDB0000554,Dihydroandrosterone,0.200000
2,HMDB0000546,Epietiocholanolone,0.181818


In [7]:
query = [{'range': (3.87, 3.93), 'mult': '*'},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
2,HMDB0000043,Betaine,1.500000
3,HMDB0000064,Creatine,1.500000
66,HMDB0002006,"2,3-Diaminopropionic acid",1.500000
63,HMDB0001991,7-Methylxanthine,1.500000
59,HMDB0001867,4-Aminohippuric acid,1.333333
53,HMDB0001398,Guaiacol,1.333333


In [8]:
query = [{'range': (3.87, 3.93), 'mult': 'dd'},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
66,HMDB0002006,"2,3-Diaminopropionic acid",1.500000
38,HMDB0000742,Homocysteine,1.333333
11,HMDB0000158,L-Tyrosine,1.200000
28,HMDB0000479,3-Methylhistidine,1.200000
0,HMDB0000021,Iodotyrosine,1.166667
33,HMDB0000609,DL-Dopa,1.166667


In [9]:
nmrdb[nmrdb['accession']=='HMDB0000742']

,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
1055,HMDB0000742,m03,2.14,1,"['14.86', '14.70', '7.47']",[3],m,2.06,2.22,"[2.07, 2.09, 2.1, 2.12, 2.13, 2.14, 2.15, 2.15...","[1037.1, 1045.1, 1051.7, 1058.8, 1066.2, 1068....","[0.018460648148148146, 0.028732638888888884, 0...",[3]
1415,HMDB0000742,m02,2.65,2,"['10.73', '8.12', '6.73']",[2],ddd,2.59,2.72,"[2.6, 2.62, 2.63, 2.64, 2.64, 2.65, 2.66, 2.66...","[1300.1, 1307.2, 1313.7, 1320.7, 1321.7, 1324....","[0.029597332649397283, 0.036881251602975125, 0...",[2]
2231,HMDB0000742,m01,3.87,1,"['7.13', '5.62']",[4],dd,3.83,3.90,"[-0.01001467351430696, -1.4673514306728919e-05...","[1928.4, 1934.0, 1935.5, 1941.1]","[0.23097445474554795, 0.2749282998732742, 0.26...",[4]


In [10]:
# Looking for Leucine
query = [{'range': (0.94, 0.99), 'mult': 't', 'ppm': np.array([0.949542,0.96010, 0.970836]), 'heights': np.array([0.25,0.5,0.25])},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
12,HMDB0000452,L-alpha-Aminobutyric acid,2.241946
22,HMDB0000687,L-Leucine,2.154733
21,HMDB0000650,D-alpha-Aminobutyric acid,1.754231
34,HMDB0001987,2-Hydroxy-2-methylbutyric acid,1.712065
8,HMDB0000339,2-Methylbutyrylglycine,1.563549
31,HMDB0001388,alpha-Linolenic acid,1.392834


In [11]:
# Looking for Threonine
query = [{'range': (4.22, 4.28), 'mult': '*', 'ppm': np.array([4.2351,4.243,4.2461,4.254 ,4.2571,4.265 ,4.2681,4.276]), 'heights': np.array([1,1,3,3,3,3,1,1]) / np.sum([1,1,3,3,3,3,1,1])},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

/Users/andres/Documents/git/github/phenocare/hmdb_utils/src/hmdb_local_tools.py:152: RuntimeWarning: invalid value encountered in scalar divide
  cm_test = np.sum(test_ppm * test_heights) / np.sum(test_heights)


,accession,name,similarity
2,HMDB0000167,L-Threonine,2.232421
5,HMDB0000244,Riboflavin,2.166881
4,HMDB0000217,NADP,1.968235
14,HMDB0000565,Galactonic acid,1.909901
21,HMDB0000982,5-Methylcytidine,1.905070
25,HMDB0001563,1-Methylguanosine,1.892107


In [12]:
# Looking for Tyrosine
query = [{'range': (7.16, 7.6), 'mult': '*', 'ppm': [7.185000,7.189691,7.192445,7.200093,7.203968,7.208965], 'heights': [0.14,1,0.3,0.28,0.92,0.14]},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
4,HMDB0000158,L-Tyrosine,2.297146
103,HMDB0004811,"2,4-Dichlorophenol",2.268032
109,HMDB0005794,Quercetin,2.252696
6,HMDB0000205,Phenylpyruvic acid,2.178898
98,HMDB0003312,Daidzein,2.154630
85,HMDB0002055,o-Cresol,2.142429


In [42]:
# Looking for Phenylalanine
query = [{'range': (7.2, 7.5), 'mult': '*', 'ppm': [7.414916,7.418995,7.421035,7.429193,7.431233,7.434292,7.440411,7.442960,7.4450005], 'heights': [0.16,0.8,0.2,0.53,1,0.2,0.13,0.33,0.2]},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
22,HMDB0000684,L-Kynurenine,2.466096
80,HMDB0004812,"2,5-Furandicarboxylic acid",2.396479
60,HMDB0001942,Phenylpropanolamine,2.301726
5,HMDB0000228,Phenol,2.286237
24,HMDB0000715,Kynurenic acid,2.238970
3,HMDB0000205,Phenylpyruvic acid,2.234585


In [43]:
# Looking for Lactate
query = [{'range': (4, 4.2), 'mult': 'q', 'ppm': [4.095,4.1065,4.118,4.1295], 'heights': [0.33,1,1,0.33]},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
24,HMDB0000190,L-Lactic acid,2.205696
14,HMDB0000125,Glutathione,1.832804
105,HMDB0005000,Loratadine,1.816799
22,HMDB0000174,L-Fucose,1.748530
11,HMDB0000101,Deoxyadenosine,1.740319
101,HMDB0003072,Quinic acid,1.375292


In [ ]:
nmrdb[nmrdb['accession'] == 'HMDB0000190']

In [44]:
# Looking for Lactate
query = [{'range': (4, 4.2), 'mult': 'q', 'ppm': [4.095,4.1065,4.118,4.1295], 'heights': [0.33,1,1,0.33]},
         {'range': (1.25, 1.36), 'mult': 'd', 'ppm': [1.31, 1.324]	, 'heights': [0.49952454832090487, 0.5004754516790951]	}]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
4,HMDB0000190,L-Lactic acid,2.852848
3,HMDB0000174,L-Fucose,1.822182
16,HMDB0005000,Loratadine,1.015542
0,HMDB0000030,Biotin,0.824622
5,HMDB0000424,2-Hydroxydecanedioic acid,0.400000
9,HMDB0000701,Hexanoylglycine,0.400000


In [ ]:
nmrdb[nmrdb['accession'] == 'HMDB0000174']

In [45]:
# Looking for Ibupeofen
query = [{'range': (7.07, 7.35), 'mult': '*', 'ppm': [7.122, 7.136, 7.241, 7.254], 'heights': [0.8,1,1,0.8]},]
     #   {'range': (0.88, 0.96), 'mult': 'd', 'ppm': [0.916, 0.927], 'heights': [1.0,1.0]}]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
86,HMDB0004812,"2,5-Furandicarboxylic acid",2.000000
72,HMDB0002085,Syringic acid,1.500000
64,HMDB0001928,Hydrochlorothiazide,1.500000
33,HMDB0000764,Hydrocinnamic acid,1.440424
0,HMDB0000020,p-Hydroxyphenylacetic acid,1.333333
21,HMDB0000669,ortho-Hydroxyphenylacetic acid,1.333333


In [ ]:
nmrdb[nmrdb['accession'] == 'HMDB0000667']

In [47]:
# Looking for Ibupeofen
query = [{'range': (7.07, 7.18), 'mult': 'd', 'ppm': [7.1228, 7.136], 'heights': [0.83,1]},
        {'range': (7.2, 7.35), 'mult': 'd', 'ppm': [7.241, 7.2542], 'heights': [1, 0.83]}]
     #   {'range': (0.88, 0.96), 'mult': 'd', 'ppm': [0.916, 0.927], 'heights': [1.0,1.0]}]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
9,HMDB0001925,Ibuprofen,1.923175
13,HMDB0005000,Loratadine,1.489583
7,HMDB0001868,5-Methoxysalicylic acid,1.323297
6,HMDB0001713,m-Coumaric acid,1.222001
1,HMDB0000259,Serotonin,1.158999
5,HMDB0000955,Isoferulic acid,0.900000


In [ ]:
nmrdb[nmrdb['accession'] == 'HMDB0001925']
#[HMDB0001872, HMDB01872, HMDB01925]

In [53]:
df[df['accession']=='HMDB0001925']['description'].values

array(['Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID) widely marketed under various trademarks including Act-3, Advil, Brufen, Motrin, Nuprin, and Nurofen. It is used for relief of symptoms of arthritis, primary dysmenorrhoea, and fever; Ibuprofen is an NSAID which is believed to work through inhibition of cyclooxygenase (COX), thus inhibiting prostaglandin synthesis. There are at least 2 variations of cyclooxygenase (COX-1 and COX-2), ibuprofen inhibits both COX-1 and COX-2. It appears that its analgesic, antipyretic, and anti-inflammatory activity are achieved principally through COX-2 inhibition; whereas COX-1 inhibition is responsible for its unwanted effects on platelet aggregation and the GI mucosa. As with other NSAIDs, ibuprofen inhibits platelet aggregation, but is not used therapeutically for this action since it is a minor and reversible effect. -- Wikipedia.'],
      dtype=object)

In [ ]:
# Looking for Ethyl vinyl ether
query = [{'range': (3.72, 3.82), 'mult': 'q', 'ppm': [3.7440, 3.7620, 3.7799, 3.7969], 'heights': [1.0, 3.0, 3.0, 1.0]},
         {'range': (1.25, 1.36), 'mult': 'd', 'ppm': [1.2903, 1.3081, 1.3256]	, 'heights': [1.0, 2.0, 1.0]}]

result = multiple_query(query, nmrdb, df)
result.head(6) 